In [34]:
from langchain.chat_models import init_chat_model

llm = init_chat_model("gpt-5")

In [20]:
llm.invoke("Hi")

AIMessage(content='Hi there! Nice to meet you. What would you like to do today? I can help answer questions, explain concepts, assist with writing or code, brainstorm ideas, plan something, translate, or just chat about a topic you’re curious about. Tell me what you need.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 385, 'prompt_tokens': 7, 'total_tokens': 392, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 320, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-5-nano-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-CvI3kzZMxooYSYFlxc5f5x4BZgjyG', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--019b974f-d91f-7df1-af44-311b23be0178-0', tool_calls=[], invalid_tool_calls=[], usage_metadata={'input_tokens': 7, 'output_tokens': 385, 'total_tokens': 

In [23]:
from langchain.tools import tool
from pydantic import BaseModel

@tool
def write_email(to:str, subject:str, content:str):
    """Write and send an email"""
    return f"Email has been send to {to} with subject : {subject} and content : {content}"

@tool
def schedule_meeting(attendies:list[str], subject:str, duration_minutes:str, preferred_day:str,  start_time:str):
    """Schdule a calender meeting"""
    date_str = preferred_day.strftime("%A, %B %d, %Y")
    return f"Meeting '{subject}' scheduled on {date_str} at {start_time} for {duration_minutes} minutes with {len(attendees)} attendees"

@tool
def check_calendar_availability(day: str) -> str:
    """Check calendar availability for a given day."""
    # Placeholder response - in real app would check actual calendar
    return f"Available times on {day}: 9:00 AM, 2:00 PM, 4:00 PM"

@tool
class Done(BaseModel):
      """E-mail has been sent."""
      done: bool
    

In [25]:
from langgraph.graph import START, END, MessagesState, StateGraph

class State(MessagesState):
    email_input:dict
    classification_decision: Literal["ignore", "respond", "notify"]

In [ ]:
from email_assistant.prompts import triage_system_prompt, triage_user_prompt, default_triage_instructions, default_background
from rich.markdown import Markdown
from pydantic import BaseModel, Field
from email_assistant.utils import parse_email, format_email_markdown


Markdown(default_background)

I'm Lance, a software engineer at LangChain.

In [39]:
class RouterSchema(BaseModel):
    """Analyze the unread email and route it according to its content."""

    reasoning: str = Field(
        description="Step-by-step reasoning behind the classification."
    )
    classification: Literal["ignore", "respond", "notify"] = Field(
        description="The classification of an email: 'ignore' for irrelevant emails, "
        "'notify' for important information that doesn't need a response, "
        "'respond' for emails that need a reply",
    )


llm_with_router = llm.with_structured_output(RouterSchema)


In [40]:
from email_assistant.utils import parse_email, format_email_markdown

llm_with_router.invoke("hello")

RouterSchema(reasoning='The message is just a greeting without any substantive information. It likely indicates the sender wants to start a conversation, so a reply asking for details or offering help is appropriate.', classification='respond')

In [ ]:
def traige_router(state):
    